In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pdb
import numpy as np
import matplotlib.pyplot as plt
import scipy 
import time
import glob
import pickle
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

In [4]:
import itertools
from sklearn.model_selection import KFold

In [5]:
sys.path.append('/home/akumar/nse/neural_control')
from utils import apply_df_filters, calc_loadings

### Initial Sabes decoding

In [6]:
with open('/home/akumar/nse/neural_control/data/sabes_decoding_df.dat', 'rb') as f:
    sabes_df = pickle.load(f)

/home/akumar/anaconda3/envs/dyn/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LinearRegression from version 0.24.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
# Approach (1): Calculate decoder on segmented version of the time series
# Approach (2): Use whole series decoder and measure its performance on different segmentations of the time series

Index(['dim', 'fold_idx', 'train_idxs', 'test_idxs', 'dimreduc_method',
       'dimreduc_args', 'coef', 'score', 'bin_width', 'filter_fn',
       'filter_kwargs', 'boxcox', 'spike_threshold', 'dim_vals', 'n_folds',
       'data_file', 'decoder', 'decoder_args', 'decoder_obj', 'r2'],
      dtype='object')

In [1]:
# Looking at Approach 2

In [ ]:
data_files = np.unique(sabes_df['data_file'].values)
folds = np.arange(5)
dimreduc_methods = dimreduc_methods = ['PCA', 'LQGCAtrace']
KCA_dimreduc_args =  [{'T':3, 'causal_weights':(1, 0), 'n_init':5}, {'T':3, 'causal_weights':(0, 1), 'n_init':5}, {'T':3, 'causal_weights':(1, 1), 'n_init':5}]
LQGCA_dimreduc_args = [{'T':3, 'loss_type':'trace', 'n_init':5}, {'T':3, 'loss_type':'fro', 'n_init':5}, {'T':3, 'loss_type':'logdet', 'n_init':5},
                       {'T':3, 'loss_type':'additive', 'n_init':5}]

dimvals = np.unique(sabes_df['dim'].values)

# Pick one
decoder_arg = sabes_df.iloc[0]['decoder_args']
df = apply_df_filters(sabes_df, decoder_args=decoder_arg)

scores = np.zeros((data_files.size, len(dimreduc_methods), folds.size, dimvals.size))

for i, data_file in enumerate(data_files):
    for j, dr_method in enumerate(dimreduc_methods):
        for f, fold in enumerate(folds):
            for d, dimval in enumerate(dimvals):            
                if 'KCA' in dr_method:
                    df_ = apply_df_filters(df, data_file=data_file, dimreduc_method='KCA', fold_idx=fold, dim=dimval)
                    # Further filter by dimreduc_args
                    if dr_method == 'KCAfwd':
                        df_ = apply_df_filters(df_, dimreduc_args=KCA_dimreduc_args[0])
                    elif dr_method == 'KCArev':
                        df_ = apply_df_filters(df_, dimreduc_args=KCA_dimreduc_args[1])
                    elif dr_method == 'KCAeven':
                        df_ = apply_df_filters(df_, dimreduc_args=KCA_dimreduc_args[2])
                elif 'LQGCA' in dr_method:
                    df_ = apply_df_filters(df, data_file=data_file, dimreduc_method='LQGCA', fold_idx=fold, dim=dimval)
                    # Further filter by dimreduc_args
                    if dr_method == 'LQGCAtrace':
                        df_ = apply_df_filters(df_, dimreduc_args=LQGCA_dimreduc_args[0])
                    elif dr_method == 'LQGCAfro':
                        df_ = apply_df_filters(df_, dimreduc_args=LQGCA_dimreduc_args[1])
                else:
                    df_ = apply_df_filters(df, data_file=data_file, dimreduc_method=dr_method, fold_idx=fold, dim=dimval)
